In [32]:
import numpy as np
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Image

# Constants
k_b = 1.380649e-23      # m^2 kg s^-2 K^-1
epsilon_0 = 8.854e-12  # Vacuum permittivity (F/m)
hbar = 1.054e-34       # Reduced Planck's constant (J·s)
mu_13 = 1.932e-29      # Dipole moment between states |1> and |3> (C·m)
mu_23 = 2.53e-29       # Dipole moment between states |2> and |3> (C·m)
N = 3.5e10             # Atomic number density (atoms/m^3)
T = 100                # In Kelvin
m = 1.41e-25           # In kg

x_steps = 100
v_th = np.sqrt(2 * k_b * T / m)
print(v_th)
x_lim = 5 * v_th
v_values = np.linspace(-x_lim, x_lim, x_steps)

# Maxwell-Boltzmann distribution weights
weights = np.sqrt(m / (2 * np.pi * k_b * T)) * np.exp(-m * v_values**2 / (2 * k_b * T))

print(np.sum(weights)*x_lim*2/x_steps)

Gamma3 = 2 * np.pi * 6   # (MHz)
Gamma31 = 5/9 * Gamma3   # Decay from level 3 to level 1 (MHz)
Gamma32 = Gamma23 = 4/9 * Gamma3   # Decay from level 3 to level 2 (MHz)
Gamma12 = Gamma21 = 0.0000001 * Gamma3    # Decay rates between levels 1 and 2 (MHz)
gamma13 = (Gamma3 + Gamma12)/2  # Decoherence rate between levels 1 and 3 (MHz)
gamma23 = (Gamma3 + Gamma21)/2  # Decoherence rate between levels 2 and 3 (MHz)
gamma12 = (Gamma12 + Gamma21)/2     # Decoherence rate between ground states (MHz)

w_1 = 2 * np.pi * 3e8 / 795e-9
k_0 = w_1 / 3e8
L = 0.1        # Length in meters
z = 0.00001    # Step size in meters

Transparency_Omega_1 = None
Transparency_Omega_2 = None
Om_2_list_adapted = None

139.9416798486614
0.9899999999991316


# Parallel Beams

To account for Doppler shifts with parallel beams, we calculate the shifted detunings as follows:


$$
\Delta_1' = \Delta_1 + k_0 \cdot v
$$

$$
\Delta_2' = \Delta_2 + k_0 \cdot v
$$

In [33]:
# Backend function to calculate OD based on user input
def calculate_OD_1(Om_1=1.0, delta_1=0.0, delta_2=0.0):
    n_rabi = 15
    Om_2_list_adapted = np.linspace(0.005, 8, n_rabi, dtype=complex)  # Coupling Rabi frequency range (dimensionless)
    Om_2_list = Om_2_list_adapted * Gamma3
    Om_1_list = np.full(n_rabi, Om_1 * Gamma3, dtype=complex)
    
    nz = int(L / z)  # Number of z steps
    z_array = np.arange(nz) * z  # Array of z values
    
    # Initialize arrays to store Rabi frequencies at each z step
    Om_2_vs_z = []
    Om_1_vs_z = []

    # Initialize arrays to store rho31 and rho32
    rho31_values = np.zeros((n_rabi, x_steps), dtype=complex)
    rho32_values = np.zeros((n_rabi, x_steps), dtype=complex)

    for i in range(nz):
        #if i == nz/4 or i == nz/2 or i == 3*nz/4: 
        print(f"Processing z step {i}/{nz}")
        
        for k in range(len(v_values)):       
            #if k == len(v_values)/4 or k == len(v_values)/2 or k == 3*len(v_values)/4: 
                #print(f"Processing k step {k}/{len(v_values)}")
            for j in range(1, len(Om_2_list)):
                Om_2_actual = Om_2_list[j]      # Convert to actual Omega_2 in MHz
                Om_1_actual = Om_1_list[j]      # Convert Om_1 to MHz

                delta_1_actual = delta_1 * Gamma3 + k_0 * v_values[k] / (2 * np.pi * 10**6)
                delta_2_actual = delta_2 * Gamma3 + k_0 * v_values[k] / (2 * np.pi * 10**6)
                
                #print(delta_1_actual)
                #print(delta_2_actual)

                # Construct A and b as per Code2
                # Define the imaginary unit
                I = 1j

                # Construct the matrix A
                A = np.array([
                    # Row 1
                    [Gamma31 + Gamma12, 0, I*Om_1_actual/2, 0, Gamma31 - Gamma21, 0, -I*np.conjugate(Om_1_actual)/2, 0],
                    # Row 2
                    [0, gamma12 - I*(delta_2_actual - delta_1_actual), I*Om_2_actual/2, 0, 0, 0, 0, -I*np.conjugate(Om_1_actual)/2],
                    # Row 3
                    [I*np.conjugate(Om_1_actual), I*np.conjugate(Om_2_actual)/2, gamma13 + I*delta_1_actual, 0, I*np.conjugate(Om_1_actual)/2, 0, 0, 0],
                    # Row 4
                    [0, 0, 0, gamma12 + I*(delta_2_actual - delta_1_actual), 0, I*Om_1_actual/2, -I*np.conjugate(Om_2_actual)/2, 0],
                    # Row 5
                    [Gamma32 - Gamma12, 0, 0, 0, Gamma32 + Gamma21, I*Om_2_actual/2, 0, -I*np.conjugate(Om_2_actual)/2],
                    # Row 6
                    [I*np.conjugate(Om_2_actual)/2, 0, 0, I*np.conjugate(Om_1_actual)/2, I*np.conjugate(Om_2_actual), gamma23 + I*delta_2_actual, 0, 0],
                    # Row 7
                    [-I*Om_1_actual, 0, 0, -I*Om_2_actual/2, -I*Om_1_actual/2, 0, gamma13 - I*delta_1_actual, 0],
                    # Row 8
                    [-I*Om_2_actual/2, -I*Om_1_actual/2, 0, 0, -I*Om_2_actual, 0, 0, gamma23 - I*delta_2_actual]
                ], dtype=complex)

                # Construct the vector b
                b = np.array([
                    Gamma31,
                    0,
                    I*np.conjugate(Om_1_actual)/2,
                    0,
                    Gamma32,
                    I*np.conjugate(Om_2_actual)/2,
                    -I*Om_1_actual/2,
                    -I*Om_2_actual/2
                ], dtype=complex)

                # Solve the linear system A * v = b
                try:
                    v = np.linalg.solve(A, b)
                except np.linalg.LinAlgError:
                    # Handle singular matrix
                    print("nan")
                    rho11_values[i][j] = np.nan
                    rho22_values[i][j] = np.nan
                    rho33_values[i][j] = np.nan
                    rho31_values[i][j] = np.nan
                    rho32_values[i][j] = np.nan
                    continue

                # Extract the solutions
                rho11 = np.real(v[0])
                rho12 = v[1]
                rho13 = v[2]
                rho21 = v[3]
                rho22 = np.real(v[4])
                rho23 = v[5]
                rho31 = v[6]
                rho32 = v[7]

                # Compute rho33
                rho33 = 1 - rho11 - rho22

                # Store the populations
                rho31_values[j][k] = rho31
                rho32_values[j][k] = rho32      
                
        # Compute the weighted sums using dot product
        rho32 = np.dot(rho32_values, weights)
        rho31 = np.dot(rho31_values, weights)
        
        #print(rho32)
        
        # Update the Rabi frequencies
        Om_2_list = Om_2_actual + z * (1j) * (k_0 / (epsilon_0 * hbar)) * N * rho32 * mu_23**2
        Om_1_list = Om_1_actual + z * (1j) * (k_0 / (epsilon_0 * hbar)) * N * rho31 * mu_13**2
        
        # Record Rabi frequencies at this z step
        Om_2_vs_z.append(Om_2_list.copy())
        Om_1_vs_z.append(Om_1_list.copy())

    # Convert lists to numpy arrays
    Om_2_vs_z = np.array(Om_2_vs_z)  # Shape: (nz, n_rabi)
    Om_1_vs_z = np.array(Om_1_vs_z)
    
    return Om_2_list_adapted, Om_2_list, Om_1_list, rho31_values, rho32_values, Om_2_vs_z, Om_1_vs_z, z_array

def plot_OD(Om_1: list,delta_1: list, delta_2):
    global Transparency_Omega_1, Transparency_Omega_2, Om_2_list_adapted
    Output_Intensity_Omega_1 = []
    Output_Intensity_Omega_2 = []
    
    for i in range(len(Om_1)):
        Om_2_list_adapted, Om_2_list, Om_1_list, rho31_values, rho32_values, Om_2_vs_z, Om_1_vs_z, z_array = calculate_OD_1(Om_1[i], delta_1[i], delta_2)
    
        # Compute transparency for Omega_1 and Omega_2
        Transparency_Omega_1 = np.abs(Om_1_vs_z[-1, :])**2 / np.abs(Om_1_vs_z[0, :])**2
        Transparency_Omega_2 = np.abs(Om_2_vs_z[-1, :])**2 / np.abs(Om_2_vs_z[0, :])**2

        # Calculate output intensity based on transparency and initial input power
        #Output_Intensity_Omega_1.append(Transparency_Omega_1 * np.abs(Om_1_vs_z[0, :])**2 / (Gamma3**2))
        Output_Intensity_Omega_1.append(Transparency_Omega_1)
        Output_Intensity_Omega_2.append(Transparency_Omega_2 * np.abs(Om_2_vs_z[0, :])**2 / (Gamma3**2))
    

    # Plot Output Intensity of Omega_1 vs Initial Omega_2
    plt.figure(figsize=(8, 6))
    for i in range(len(Om_1)):
        plt.plot(np.real(Om_2_list_adapted)**2, Output_Intensity_Omega_1[i], label=f'$\Omega_1$ = {Om_1[i]}, $\Delta_1$ = {delta_1[i]}')
        
    plt.xlabel('Initial Intensity 2 ($I_{in} / (\\Gamma_3)^2$)')
    plt.ylabel('Output Intensity 1 ($I_{out} / I_{in}$)')
    plt.title(f'Output Intensity 1 vs $I_{{2,\t{{in}}}} / (\\Gamma_3)^2$ for 'f'$\\Delta_2/\\Gamma_3={delta_2}$')
    plt.grid(True)
    plt.legend()
    plt.show()

    # Plot Output Intensity of Omega_2 vs Initial Omega_2
    plt.figure(figsize=(8, 6))
    for i in range(len(Om_1)):
        plt.plot(np.real(Om_2_list_adapted)**2, Output_Intensity_Omega_2[i], label=f'$\Omega_1$ = {Om_1[i]}, $\Delta_1$ = {delta_1[i]}')
    plt.xlabel('Initial Intensity 2 ($I_{in} / (\\Gamma_3)^2$)')
    plt.ylabel('Output Intensity 2 ($I_{out} / (\\Gamma_3)^2$)')
    plt.title(f'Output Intensity 2 vs $I_{{2,\t{{in}}}} / (\\Gamma_3)^2$ for 'f'$\\Delta_2/\\Gamma_3={delta_2}$')
    plt.grid(True)
    plt.legend()
    plt.show()


# Create the button to start the simulation
run_button = widgets.Button(
    description="Run Simulation",
    button_style='success',  # 'success', 'info', 'warning', 'danger' or '' (default)
    tooltip='Click to run the simulation',
    icon='play'
)

# Function to be triggered when the button is clicked
def run_simulation(b):
    # Extract the current slider values
    Om_1 = [1]
    delta_1 = [0.25]
    delta_2 = 0
    
    # Call the plot_OD function with these values
    plot_OD(Om_1, delta_1, delta_2)

# Attach the function to the button
run_button.on_click(run_simulation)

# Display the sliders and the button
display(run_button)


Button(button_style='success', description='Run Simulation', icon='play', style=ButtonStyle(), tooltip='Click …

Processing z step 0/10000
Processing z step 1/10000
Processing z step 2/10000
Processing z step 3/10000
Processing z step 4/10000
Processing z step 5/10000
Processing z step 6/10000
Processing z step 7/10000
Processing z step 8/10000
Processing z step 9/10000
Processing z step 10/10000
Processing z step 11/10000
Processing z step 12/10000
Processing z step 13/10000
Processing z step 14/10000
Processing z step 15/10000
Processing z step 16/10000
Processing z step 17/10000
Processing z step 18/10000
Processing z step 19/10000
Processing z step 20/10000
Processing z step 21/10000
Processing z step 22/10000
Processing z step 23/10000
Processing z step 24/10000
Processing z step 25/10000
Processing z step 26/10000
Processing z step 27/10000
Processing z step 28/10000
Processing z step 29/10000
Processing z step 30/10000
Processing z step 31/10000
Processing z step 32/10000
Processing z step 33/10000
Processing z step 34/10000
Processing z step 35/10000
Processing z step 36/10000
Processing 